#### 导入相关包 

In [1]:
from datasets import *

#### 加载数据集

In [2]:
dataset = load_dataset("csv", data_dir="../all_data/csv", split="train")
# dataset = load_dataset("csv",data_files="../all_data/电力现货市场百问百答.csv",split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 240
})

In [1]:
import pandas as pd
import os

# 假设Excel文件都在一个名为'data'的文件夹下
folder_path = '../all_data/excel'
excel_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xlsx')]

data_frames = []
for file in excel_files:
    df = pd.read_excel(file)
    data_frames.append(df)

combined_data = pd.concat(data_frames, ignore_index=True)

In [2]:
combined_data

,question,answer
0,1、什么是光伏发电？什么是分布式光伏系统？,光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能...
1,2、什么是户用分布式光伏系统？,户用分布式光伏系统是指利用自然人宅基地范围内的建筑物，比如自有住宅，以及附属物建设的分布...
2,3、您知道光伏发电的历史起源吗？,1839 年，19 岁的法国贝克勒尔做物理实验时，发现在导电液中的两种金属电极用光照射时...
3,4、光伏电池是怎么发电的？,光伏电池是一种具有光－电转换特性的半导体器件，它直接将太阳辐射能转换成直流电，是光伏发电...
4,5、户用分布式光伏发电系统由哪些部件构成？,户用分布式光伏发电系统由光伏方阵（光伏方阵由光伏组件串并联而成)、光伏逆变器、光伏支架、...
...,...,...
407,169.什么是电力系统的运行操作？,电力系统中的设备有运行、热备用、冷备用和检修四种不同的状态，运行操作是指变更电力系统设备...
408,170.倒闸操作的分类有哪些？有哪些要求？,倒闸操作从满足新技术应用和经济性，即减人增效提高劳动生产率的要求来考虑，倒闸操作可分为就...
409,171.什么是遥控操作和程序操作？有哪些要求？,遥控操作是指从调度端或集控站发出远方操作指令，以微机监控系统或变电站的RTU当地功能为技...
410,172.倒闸操作的基本要求是什么？包括哪些内容？,倒闸操作是一项严谨的工作，应根据值班调度员或运行值班负责人的指令，在受令人复诵无误后执行...


#### 按照Qwen 中 Lora要求的数据格式构造训练的数据集

In [3]:
train_data= []

for index ,(q,a) in enumerate(zip(combined_data['question'], combined_data['answer'])):
    qa_json = {}
    qa_json['id'] = "identity_"+str(index)
    qa_json['conversations'] = []
    qa_json['conversations'].append({
                         "from": "user",
                         "value": q
                         })
    qa_json['conversations'].append({
                         "from": "assistant",
                         "value": a
                         })
    train_data.append(qa_json)

train_data[0]

{'id': 'identity_0',
 'conversations': [{'from': 'user', 'value': '1、什么是光伏发电？什么是分布式光伏系统？'},
  {'from': 'assistant',
   'value': '  光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。\n  分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自\n发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。\n  分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。'}]}

In [6]:
train_data[411]

{'id': 'identity_411',
 'conversations': [{'from': 'user', 'value': '173.倒闸操作包括哪些内容？'},
  {'from': 'assistant',
   'value': '  倒闸操作的内容包括：\n1）电力线路停送电操作。\n2）电力变压器停送电操作。\n3）电网合环或解环操作。\n4）倒换母线操作。\n5）旁路母线代路操作。\n6）中性点接地方式改变和消弧线圈分头调整。'}]}

#### 将数据集导出为json格式

In [7]:
import json

# 导出JSON文件
with open("train.json", "w", encoding='utf-8') as file:
    json.dump(train_data, file,indent=4, ensure_ascii=False)

#### 导出测试数据

In [11]:
question = list(combined_data['question'])
answers = list(combined_data['answer'])
# print(len(question), len(answers))
QA_list = []

for q,a in zip(question, answers):
    qa = {'question': q, 'answer': a}
    QA_list.append(qa)

In [12]:
QA_list[0]

{'question': '1、什么是光伏发电？什么是分布式光伏系统？',
 'answer': '  光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。\n  分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自\n发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。\n  分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。'}

In [13]:
import random

random_tests = random.sample(QA_list, 80)
print(random_tests)



[{'question': '45.避雷器的性能有哪些？', 'answer': '  在确定避雷器性能上，有以下几个重要关系：\n（1）从保证避雷器运行安全方面考虑，其持续运行电压应等于或大于电网的最大运行相电压。\n（2）从保证避雷器不会老化太快方面考虑，其额定电压应等于或大于电网持续运行电压的1.25倍。\n（3）避雷器在标称电流（雷电流）下的残压，等于额定电压（峰值）乘以保护比，目前避雷器的保护比约等于1.6~1.7。'}, {'question': '101.电网运行方式包括哪些内容?它与电网结构有什么关系？', 'answer': '  电网运行方式包括网内电源与负荷的电力电量平衡、主要厂站(所)的主接线方式及保护配合、各电网间的联网及联络线传输功率的控制、电网的调峰、无功电源的运行调度，以及各种负荷情况下电网的运行特性等等。\n  电网结构与它的关系是：合理的电网结构是各种运行方式的基础，它约束和规定了电网的运行方式。针对不同的电网结构和不同的运行方式，研究电网的特性，确定各种事故条件下应采取的对策，是电网运行工作的重要任务之一。'}, {'question': '25什么是差价合约？', 'answer': '  差价合约（contractfordifference，CFD）具有电力金融市场属性，其金融本质是互换，从广义上来讲，互换是要求交易双方根据预定义的交换规则，在约定的时间段内，就一系列应收现金流（如利息、价差等）进行交换的双边合\n约。在电力市场中，差价合约不涉及实际电力的交割，是一种提前确定收益的金融工具，买卖双方可以商定商品的履约价格和交易数量。签订差价合约后，买卖双方可以和其他市场主体一样参与集中市场，一旦集中市场上的交易已经完\n成，差价合约就可以按照如下的方式进行结算：\n1）如果差价合约的履约价格高于集中市场价格，购买方需要向卖出方支付一定金额，等于这两种价格的差价乘以合约规定的交易数量。\n2）如果差价合约的履约价格低于集中市场价格，卖出方需要向购买方支付一定金额，等于这两种价格的差价乘以合约规定的交易数量。\n简而言之，差价合约的售方按照合约数量支付基准价格，收取合约价格。\n  差价合约可以按照交易主体或合约价格的生效方向的不同进行分类：\n1）差价合约的交易主体包括购买方和售方，均可以是独立的市场主体或者政府。如果双方都是

In [14]:
import json

with open('../all_data/json/test.json', 'w', encoding='utf-8') as f:
    json.dump(random_tests, f, ensure_ascii=False, indent=4)